# postprocess refers_to_triplet

> 외부 법

04_regulation_main에 있는 unique 확인

In [1]:
import json

with open("../../data/DCM/DCM_json/04/04_regulation_main.json", 'r', encoding='utf-8') as f:
    data_04 = json.load(f)


len(data_04)

6173

In [2]:
document_titles = []
for row in data_04:
    document_titles.append(row['metadata']['document_title'])

document_titles = list(set(document_titles))

In [3]:
len(document_titles)

95

In [4]:
data_list_meta = {
        "「자본시장과 금융투자업에 관한 법률」" : "01_law_main",
        "「자본시장과 금융투자업에 관한 법률 시행령」" : "01_enforcement_main",
        "「자본시장과 금융투자업에 관한 법률 시행규칙」" : "01_order_main",
        "「금융지주회사감독규정」" : "02_law_main",
        "「금융지주회사감독규정시행령」" : "02_enforcement_main",
        "「금융지주회사감독규정시행세칙」" : "02_order_main",
        "「증권의 발행 및 공시 등에 관한 규정」" : "03_regulation_main",
        "「은행법」": "08_law_main",
        "「은행법 시행령」": "08_enforcement_main",
    }

for document_title in document_titles:
    data_list_meta["「" + document_title + "」"] = "04_regulation_main"

In [5]:
import json

with open("../../data/graph/clause/edge_triplet/04/04_refers_to_triplets.json", 'r', encoding='utf-8') as f:
    data = json.load(f)


In [6]:
data_list_count = {}
for key in data_list_meta.keys():
    data_list_count[key] = 0

1. contents에서 「」가 포함되어 있는지 확인
2. 뒤의 단어 중 제x조제x항 과 같이 있는지 확인
3. target_index가 2와 일치할 경우, target_document를 갱신하기

In [7]:
import re

import copy 


total_matches = []
# Pattern to match law names inside 「」
law_pattern = r'「(.*?)」'
# Pattern to match content between closing 」 and article number
after_law_pattern = r'\s*(?:\([^)]*\)\s*)*'

# Pattern to match article numbers after law name
article_pattern = r'(제[\d가-힣]+조(?:의[\d가-힣]+)?(?:제[\d가-힣]+항)?)'

for row in data[0]:
    text = row['contents']
    if '「' in text:
        print(f"###### text : {text} #########")

    for law_match in re.finditer(law_pattern, text):
        # Extract law name
        law_name = law_match.group(1)
        # Remove parentheses and content inside from law name
        law_name_clean = re.sub(r'\(.*?\)', '', law_name)
        # Add back the 「」 around law name
        law_name_with_brackets = f'「{law_name_clean}」'
        
        # Position after closing 」 in the text
        pos_after_law = law_match.end()
        
        # Skip any content inside parentheses and spaces
        after_law_match = re.match(after_law_pattern, text[pos_after_law:])
        if after_law_match:
            pos_after_skipped = pos_after_law + after_law_match.end()
        else:
            pos_after_skipped = pos_after_law
        
        # Search for article number starting from pos_after_skipped
        article_match = re.match(article_pattern, text[pos_after_skipped:])
        if article_match:
            # Extract article number
            article_number = article_match.group(1)
            
            # 제21조의2에 와 같은 경우 '에' 조사 제거
            article_number = article_number.replace("에", "")

            # Create tuple and add to list
            total_matches.append((law_name_with_brackets, article_number))
            print('## law name :', law_name_with_brackets, 'article number :', article_number)

            if article_number == row['target_index']:
                if law_name_with_brackets in data_list_meta.keys():
                    # print(f"## target doc changed {row['target_document']} -> {data_list_meta[law_name_with_brackets]}")
                    print(f"## target original {row['target_document']}")
                    row['target_document'] = data_list_meta[law_name_with_brackets]
                    data_list_count[law_name_with_brackets] += 1
                    print(f"## target changed {row['target_document']}")


                else:
                    print(f"## target doc {law_name_with_brackets} not in our documents!\n\n")
                    print(f"## target original {row['target_document']}")
                    row['target_document'] = "not_in_my_document"
                    print(f"## target changed {row['target_document']}")
                    
    

###### text : 이 규준은 「자본시장과 금융투자업에 관한 법률」(이하 "자본시장법"이라 한다) 제8조에 따른 금융투자업자(겸영금융투자업자 및 집합투자업자는 제외한다) 중 투자매매업 또는 투자중개업을 영위하는 금융투자회사(이하 "회사"라 한다)가 투자자예탁금이용료를 투자자에게 지급함에 있어 이용료율을 합리적으로 산정하는데 필요한 사항을 정함을 목적으로 한다. #########
## law name : 「자본시장과 금융투자업에 관한 법률」 article number : 제8조
## target original 04_regulation_main
## target changed 01_law_main
###### text : 위원회는「고난도 금융투자상품 심사위원회 설치 및 운영에 관한 규정(이하 "심사위원회 규정"이라 한다)」제2조에 따라 상품분류점검위원회(이하 "점검위원회"라 한다)를 둔다. #########
## law name : 「고난도 금융투자상품 심사위원회 설치 및 운영에 관한 규정」 article number : 제2조
## target doc 「고난도 금융투자상품 심사위원회 설치 및 운영에 관한 규정」 not in our documents!


## target original not_in_my_document
## target changed not_in_my_document
###### text : 최근 5년간 관련법규 또는 금융관련 법령을 위반하여 「금융회사의 지배구조에 관한 법률」 제26조제1항제1호의 요건을 충족하지 못하는 경우 그 직을 상실하며, 회사는 기타 사유로 준법감시인을 해임하고자 하는 경우 이사회의 의결을 거쳐야 한다. #########
## law name : 「금융회사의 지배구조에 관한 법률」 article number : 제26조제1항
## target doc 「금융회사의 지배구조에 관한 법률」 not in our documents!


## target original not_in_my_document
## target 

In [8]:
data[0]

[{'current_document': '04_regulation_main',
  'current_index': '제2조제2항',
  'target_document': '04_regulation_main',
  'target_index': '제29조',
  'contents': '이 가이드라인은 전자금융감독규정 제29조의 적용을 받는 프로그램을 대상으로 한다.'},
 {'current_document': '04_regulation_main',
  'current_index': '제5조제6항',
  'target_document': '04_regulation_main',
  'target_index': '제4조',
  'contents': '회사는 제4조에 따른 기본예탁금을 납부하지 않는 고객의 주문에 대하여 수탁을 거부하여야 한다.'},
 {'current_document': '04_regulation_main',
  'current_index': '제7조제4항',
  'target_document': '04_regulation_main',
  'target_index': '제7조제1항',
  'contents': '제1항에 따라 사전안내를 받은 고객이 방문판매등을 거절하거나, 방문판매등을 실시할 시간·장소 등을 변경하고자 하는 경우 회사는 이를 따라야 한다.'},
 {'current_document': '04_regulation_main',
  'current_index': '제7조제5항',
  'target_document': '04_regulation_main',
  'target_index': '제9조제1항',
  'contents': '회사는 예탁금이용료의 지급기준이 변경되는 경우 제9조제1항제2호에 따라 매매거래 등을 통지할 때 그 변경내용을 함께 가입자에게 알려준다.'},
 {'current_document': '04_regulation_main',
  'current_index': '제9조제2항',
  'target_document': '04_r